# NO WORKING WELL

# Link kaggle to Google Cloud
https://www.youtube.com/watch?v=xP99eh6nQN0&feature=youtu.be

In [45]:

# Set your own values for these. bucket_name should be the project_id + '-lcm'.
PROJECT_ID = 'automl-disastertweets'
bucket_name = 'automl-disastertweets-lcm'

region = 'us-central1' # Region must be us-central1
dataset_display_name = 'kaggle_tweets'
model_display_name = 'kaggle_starter_model1'

storage_client = storage.Client(project=PROJECT_ID)

# adding ClientInfo here to get the gapic_v1 call in place
client = automl.AutoMlClient(client_info=ClientInfo())

print(f'Starting AutoML notebook at {datetime.fromtimestamp(time.time()).strftime("%Y-%m-%d, %H:%M:%S UTC")}')

Starting AutoML notebook at 2020-02-24, 15:08:26 UTC


In [46]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
from datetime import datetime

from sklearn.model_selection import train_test_split

from google.cloud import storage
from google.cloud import automl_v1beta1 as automl

# workaround to fix gapic_v1 error
from google.api_core.gapic_v1.client_info import ClientInfo


In [47]:
# AUTOMLWrapper copied from here: https://www.kaggle.com/philculliton/automlwrapper

class AutoMLWrapper():
    def __init__(self, client, project_id, bucket_name, region='us-central1', dataset_display_name=None, model_display_name=None):
        self.client = client
        self.project_id = project_id
        self.bucket_name = bucket_name
        self.region = region
        self._location_path = self.client.location_path(self.project_id, self.region)
        
        self.dataset = None
        self.dataset_id = None
        self.dataset_display_name = dataset_display_name
        
        self.model = None
        self.model_full_path = None
        self.model_display_name = model_display_name
        
    
    def set_dataset(self, dataset):
        self.dataset = dataset
        self.dataset_id = dataset.name.split('/')[-1]
        self.dataset_display_name = dataset.display_name
        
    def create_dataset(self, classification_type='MULTICLASS', dataset_display_name=None):
        if not classification_type:
            classification_type = 'MULTICLASS'
        dataset_metadata = {'classification_type': classification_type}
        
        if dataset_display_name:
            self.dataset_display_name = dataset_display_name
            
        if not (dataset_display_name or self.dataset_display_name):
            self.dataset_display_name = f'automldataset_{datetime.fromtimestamp(time.time()).strftime("%Y%m%d%H%M%S")}'
            
        dataset_info = {
            'display_name': self.dataset_display_name,
            'text_classification_dataset_metadata': dataset_metadata,
        }
        print(f'making new dataset, display name: {self.dataset_display_name}')
        dataset = self.client.create_dataset(self._location_path, dataset_info)
        self.set_dataset(dataset)
        
        print(dataset)
        return dataset
        
    def import_gcs_data(self, gcs_path):
        # Slice off leading slash
        if gcs_path[0] == '/':
            gcs_path = gcs_path[1:]
        data_config = {'gcs_source': { 'input_uris': [f'gs://{self.bucket_name}/{gcs_path}']}}

        print('importing csv data. This may take a moment')
        operation = self.client.import_data(name=self.dataset.name, input_config=data_config)
        print(operation)

        result = operation.result()
        print(result)

        return
    
    def get_dataset_by_display_name(self, dataset_display_name=None):
        if not dataset_display_name:
            dataset_display_name = self.dataset_display_name
            
        if not dataset_display_name:
            print('no dataset_display_name provided. Please set as attribute, or provide in call parameters.')
            return None
        
        filter_ = 'display_name=' + dataset_display_name
        print(f'searching for dataset named: {dataset_display_name}')
        list_datasets_response = self.client.list_datasets(self._location_path, filter_=filter_)

        matching_datasets = []
        for d in list_datasets_response:
            matching_datasets.append(d)
            self.set_dataset(d) # set the most recent dataset as our match
        
        if not matching_datasets:
            print('no matching datasets found')
        else:
            print(f'found {len(matching_datasets)} matching datasets')
            
        return matching_datasets
    
    def set_model(self, model):
        self.model = model
        self.model_full_path = model.name
        self.model_display_name = model.display_name
        
    def get_model_by_display_name(self, model_display_name=None):
        if not model_display_name:
            model_display_name = self.model_display_name
            
        if not model_display_name:
            print('no model_display_name provided. Please set as attribute, or provide in call parameters.')
            return None
        
        filter_ = 'display_name=' + model_display_name
        print(f'searching for model named: {model_display_name}')
        models_list = self.client.list_models(self._location_path, filter_)
        
        matching_models = []
        for model in models_list:
            matching_models.append(model)
            self.set_model(model)
            
        if not matching_models:
            print('no matching models found')
        else:
            print(f'found {len(matching_models)} matching models')
            
        return matching_models
    
    def deploy_model(self):
        if self.model.deployment_state == 2: # because undeployed == 2 for some reason
            print(f'Deploying model: {self.model_display_name} at {datetime.fromtimestamp(time.time()).strftime("%Y-%m-%d, %H:%M:%S UTC")}')
            response = self.client.deploy_model(name=self.model_full_path)
            while self.model.deployment_state == 2:
                time.sleep(120) # check every other minute if deployment is done
                self.get_model_by_display_name() # refresh the model info
            print(f'Finished deploying model: {self.model_display_name} around {datetime.fromtimestamp(time.time()).strftime("%Y-%m-%d, %H:%M:%S UTC")}')
        else:
            print(f'model {self.model_display_name} is already deployed')

    def undeploy_model(self):
        if self.model.deployment_state == 1:
            print(f'Undeploying model: {self.model_display_name} at {datetime.fromtimestamp(time.time()).strftime("%Y-%m-%d, %H:%M:%S UTC")}')
            response = self.client.undeploy_model(name=self.model_full_path)
            while self.model.deployment_state == 1:
                time.sleep(120) # check every other minute if undeployment is done
                self.get_model_by_display_name() # refresh the model info
            print(f'Finished undeploying model: {self.model_display_name} around {datetime.fromtimestamp(time.time()).strftime("%Y-%m-%d, %H:%M:%S UTC")}')
        else:
            print(f'model {self.model_display_name} is already undeployed')
        
    # my_model = amw.train_model(dataset.name.split('/')[-1], model_display_name, project_location)
    def train_model(self, model_display_name=None): # dataset_id, model_display_name, project_location):
        
        if model_display_name:
            self.model_display_name = model_display_name
        if not (model_display_name or self.model_display_name):
            self.model_display_name = f'automlmodel_{datetime.fromtimestamp(time.time()).strftime("%Y%m%d%H%M%S")}'
            
        print(f'making new model with dataset {self.dataset_id}, named {self.model_display_name}')

        # Set model name and model metadata for the dataset.
        model_info = {
            "display_name": self.model_display_name,
            "dataset_id": self.dataset_id,
            "text_classification_model_metadata": {},
        }

        print('creating and training model')
        # Create a model with the model metadata in the region.
        create_model_response = self.client.create_model(self._location_path, model_info)
        print(create_model_response)

        result = create_model_response.result()

        return self.get_model_by_display_name()[0]
    
    def set_prediction_client(self, prediction_client):
        self.prediction_client = prediction_client
    
    # awm.get_predictions(self, test_df, input_col_name='text', ground_truth_col_name='choose_one', threshold=0.5, limit=20)
    def get_predictions(self, test_df, input_col_name, ground_truth_col_name=None, threshold=0.5, limit=None, verbose=False):
        # score | class | text
        predictions_list = []
        correct = 0
        total_test_size = limit if limit else len(test_df)
        
        for i in range(total_test_size):
            row = test_df.iloc[i]
            snippet = row[input_col_name]
            if ground_truth_col_name:
                ground_truth = row[ground_truth_col_name]

            # Set the payload by giving the content and type of the file.
            payload = {"text_snippet": {"content": snippet, "mime_type": "text/plain"}}

            # params is additional domain-specific parameters.
            # currently there is no additional parameters supported.
            params = {}
            response = self.prediction_client.predict(self.model_full_path, payload, params)

            for result in response.payload:
        #         print("Predicted class name: {}".format(result.display_name))
        #         print("Predicted class score: {}".format(result.classification.score))
                if result.classification.score >= threshold:
                    prediction = {'score': result.classification.score, 
                                  'class': result.display_name,
                                  'text' : snippet}
                    if verbose:
                        print(f'[{i}/{total_test_size}] Predicted class: {result.display_name}, score: {result.classification.score}')

                    if ground_truth_col_name:
                        prediction['ground_truth'] = ground_truth
                        if result.display_name == ground_truth:
                            correct += 1 
                    predictions_list.append(prediction)
            time.sleep(0.3)
                    
        if ground_truth_col_name:
            acc = correct / len(predictions_list)
            print(f'accuracy is {acc}')

        predictions_df = pd.DataFrame(predictions_list)
        return predictions_df

In [48]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/train.csv


In [60]:
# READ THE TRAINING AND TESTING DATASET
train_data = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test_data = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
def callback(operation_future):
    result = operation_future.result()


# GCS upload/download utilities

In [50]:
# These functions make upload and download of files from the kernel to Google Cloud Storage easier. This is needed for AutoML

In [61]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket. https://cloud.google.com/storage/docs/ """
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print('File {} uploaded to {}'.format(
        source_file_name,
        'gs://' + bucket_name + '/' + destination_blob_name))
    
def download_to_kaggle(bucket_name,destination_directory,file_name,prefix=None):
    """Takes the data from your GCS Bucket and puts it into the working directory of your Kaggle notebook"""
    os.makedirs(destination_directory, exist_ok = True)
    full_file_path = os.path.join(destination_directory, file_name)
    blobs = storage_client.list_blobs(bucket_name,prefix=prefix)
    for blob in blobs:
        blob.download_to_filename(full_file_path)

In [62]:
bucket = storage.Bucket(storage_client, name=bucket_name)
if not bucket.exists():
    bucket.create(location=region)

In [63]:
# Select the text body and the target value, for sending to AutoML NL
train_data[['text','target']].to_csv('train.csv')

In [64]:
training_gcs_path = 'uploads/kaggle_getstarted/full_train.csv'
upload_blob(bucket_name, 'train.csv', training_gcs_path)



File train.csv uploaded to gs://automl-disastertweets-lcm/uploads/kaggle_getstarted/full_train.csv


In [65]:
# Create the class instance

amw = AutoMLWrapper(client=client, project_id = PROJECT_ID, bucket_name = bucket_name, region = 'us-central1', 
                    dataset_display_name = dataset_display_name, model_display_name = model_display_name)

In [66]:
# Create or retrieve the dataset

print(f'Getting dataset ready at {datetime.fromtimestamp(time.time()).strftime("%Y-%m-%d, %H:%M:%S UTC")}')
if not amw.get_dataset_by_display_name(dataset_display_name):
    print('dataset not found')
    amw.create_dataset()
    amw.import_gcs_data(training_gcs_path)

amw.dataset
print(f'Dataset ready at {datetime.fromtimestamp(time.time()).strftime("%Y-%m-%d, %H:%M:%S UTC")}')

Getting dataset ready at 2020-02-24, 15:20:39 UTC
searching for dataset named: kaggle_tweets
found 1 matching datasets
Dataset ready at 2020-02-24, 15:20:39 UTC


# Training of the model

In [67]:
print(f'Getting model trained at {datetime.fromtimestamp(time.time()).strftime("%Y-%m-%d, %H:%M:%S UTC")}')

if not amw.get_model_by_display_name():
    print(f'Training model at {datetime.fromtimestamp(time.time()).strftime("%Y-%m-%d, %H:%M:%S UTC")}')
    amw.train_model()

print(f'Model trained. Ensuring model is deployed at {datetime.fromtimestamp(time.time()).strftime("%Y-%m-%d, %H:%M:%S UTC")}')
amw.deploy_model()
amw.model
print(f'Model trained and deployed at {datetime.fromtimestamp(time.time()).strftime("%Y-%m-%d, %H:%M:%S UTC")}')

# ERROR 

Getting model trained at 2020-02-24, 15:20:41 UTC
searching for model named: kaggle_starter_model1
no matching models found
Training model at 2020-02-24, 15:20:41 UTC
making new model with dataset TCN3031760377080709120, named kaggle_starter_model1
creating and training model


FailedPrecondition: 400 The dataset doesn't have any examples.

### PREDICTION

In [68]:
#ERROR BECAUSE THE PREVIOUS STEP WASN'T FINISHED: DEPLOYMENT 

print(f'Begin getting predictions at {datetime.fromtimestamp(time.time()).strftime("%Y-%m-%d, %H:%M:%S UTC")}')

# Create client for prediction service.
prediction_client = automl.PredictionServiceClient()
amw.set_prediction_client(prediction_client)

predictions_df = amw.get_predictions(test_data, 
                                     input_col_name='text', 
#                                      ground_truth_col_name='target', # we don't have ground truth in our test set
                                     limit=None, 
                                     threshold=0.5,
                                     verbose=False)

print(f'Finished getting predictions at {datetime.fromtimestamp(time.time()).strftime("%Y-%m-%d, %H:%M:%S UTC")}')

Begin getting predictions at 2020-02-24, 15:33:56 UTC


InvalidArgument: 400 Request contains an invalid argument.

In [ ]:
# Undeploy the model to stop charges
amw.undeploy_model()

In [ ]:
submission_df = pd.concat([nlp_test_df['id'], predictions_df['class']], axis=1)
submission_df = submission_df.rename(columns={'class':'target'})
submission_df.head()

In [ ]:
submission_df.to_csv("submission.csv", index=False, header=True)